In [76]:
'''
CAPP 122: Project Maroon Justice Index
Jimena Salinas
Code for creating a scatter plot that
shows the relationship between period poverty
index values and the number of community 
resource centers and retailers, and a scatter
for showing the relationship between income,
number of eligible women, and index.
'''

import pandas as pd
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Data prep
os.chdir('/home/jimenas/30122-project-maroon-justice-index/ppindex/')
print(os.getcwd())

pp_dta = pd.read_json("data/pp_index.json")

census_geo0 = gpd.read_file('src/census_tract_with_neigh_names.geojson')

#covert geopandas data frame to a regular dataframe for merging
census_geo = pd.DataFrame(census_geo0)
census_geo

# # # add per capita disposable monthly income
pp_dta["avg_disposable_income_per_month"] = pp_dta["disposable_income_per_month"] / pp_dta['total_pop_in_tract']

# Rename the "tractce10" column to "tract" for consistency with the pp_dta file
census_geo['tractce10'] = census_geo['tractce10'].astype(str)
pp_dta['tract'] = pp_dta['tract'].astype(str)
census_geo.rename(columns={'tractce10': 'tract'}, inplace=True)
census_geo = census_geo.loc[:, ['tract', 'neighborhood_name']]

# # # Merge the two dataframes using the "tract" column
pp_dta = pd.merge(left=pp_dta, right=census_geo, on='tract', how='left')
pp_dta

pp_dta.columns
pp_dta = pp_dta.loc[:, ['tract', 'neighborhood_name', 'pp_index', 'disposable_income_per_month', 
                        'avg_disposable_income_per_month', 'total_eligible_women', 'total_pop_in_tract',
                        'number_of_centers']]

pp_dta.to_json("data/index_data_for_scatter.json", orient='records')

/home/jimenas/30122-project-maroon-justice-index/ppindex


In [77]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.colors as colors

pp_dta = pd.read_json("data/index_data_for_scatter.json")

# Custom colorscale with shades of dark wine red
custom_reds = colors.make_colorscale(['#FFEEEE', '#E27B91', '#C83652', '#722f37'])

# Create a scatter plot of pp_index vs. number_of_centers, where each point represents a tract
scatter = go.Scatter(x=pp_dta['number_of_centers'], y=pp_dta['pp_index'],
                     mode='markers', 
                     marker=dict(color=pp_dta['pp_index'], 
                                 colorscale=custom_reds, 
                                 reversescale=False,
                                 sizemode='diameter', 
                                 sizeref=0.1, 
                                 size=6, 
                                 opacity=1,
                                 colorbar=dict(title='Period Poverty Index')),
                                 text=pp_dta['neighborhood_name'])

fig = make_subplots(rows=1, cols=1, specs=[[{}]], shared_xaxes=True, 
                    shared_yaxes=False, horizontal_spacing=0.1)

fig.add_trace(scatter, row=1, col=1)

# y-axis title
fig.update_yaxes(title_text='Period Poverty Index', row=1, col=1, range=[0.05, None])

# x-axis title 
fig.update_xaxes(title_text='Number of Community-Based Services and Commercial Retailers', row=1, col=1)

# set title and make the background white
fig.update_layout(plot_bgcolor='white')

fig.show()



In [78]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.colors as colors

pp_dta = pd.read_json("data/index_data_for_scatter.json")

# Custom colorscale dark wine red
custom_reds = colors.make_colorscale(['#FFEEEE', '#E27B91', '#C83652', '#722f37'])

# Filter the data to show only income per month of 2,900 or less
filtered_data = pp_dta[pp_dta['avg_disposable_income_per_month'] <= 2900]

# Create a scatter plot of total_eligible_women vs. avg_disposable_income_per_month, where each point represents a tract
scatter = go.Scatter(x=filtered_data['avg_disposable_income_per_month'], y=filtered_data['total_eligible_women'],
                     mode='markers', 
                     marker=dict(color=filtered_data['pp_index'], 
                                 colorscale=custom_reds, 
                                 reversescale=False,
                                 sizemode='diameter', 
                                 sizeref=0.1, 
                                 size=filtered_data['pp_index']*6, 
                                 opacity=0.6,
                                 colorbar=dict(title='Period Poverty Index')), 
                                 text=filtered_data['neighborhood_name'])

fig = make_subplots(rows=1, cols=1, specs=[[{}]], shared_xaxes=True, 
                    shared_yaxes=False, horizontal_spacing=0.1)

fig.add_trace(scatter, row=1, col=1)

# y-axis title
fig.update_yaxes(title_text='Total Eligible Women', row=1, col=1)

# x-axis title 
fig.update_xaxes(title_text='Average Disposable Income per Month', row=1, col=1)

# set title and make the background white
fig.update_layout(plot_bgcolor='white')

fig.show()
